## Run on HARDAC

In [1]:
source ../config_duke.sh -v

You are on Duke Server: HARDAC
BASE DIRECTORY:     /gpfs/fs1/data/reddylab/Kuei
PATH OF SOURCE:     /gpfs/fs1/data/reddylab/Kuei/source
PATH OF EXECUTABLE: /gpfs/fs1/data/reddylab/Kuei/exe
PATH OF ANNOTATION: /gpfs/fs1/data/reddylab/Kuei/annotation
PATH OF PROJECT:    /gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_STARRseq/notebooks
PATH OF RESULTS:    /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect



In [2]:
ls ${FD_RES}

annotation_fragment  count_fragment  coverage  data  log  model_linear  source


In [3]:
ls ${FD_RES}/annotation_fragment

filter_motif_score095  Input4                    TFX2_Dex   TFX4_Dex
Input1                 Input5                    TFX2_DMSO  TFX4_DMSO
Input2                 target_PER1_MA0099.3.bed  TFX3_Dex   TFX5_Dex
Input3                 target_PER1_MA0113.3.bed  TFX3_DMSO  TFX5_DMSO


In [4]:
dir ${FD_RES}/annotation_fragment/Input?/chr21.bed

2.7G Sep  1 17:19 /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/annotation_fragment/Input1/chr21.bed
2.9G Sep  1 14:54 /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/annotation_fragment/Input2/chr21.bed
3.4G Sep  1 04:56 /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/annotation_fragment/Input3/chr21.bed
2.7G Sep  1 14:16 /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/annotation_fragment/Input4/chr21.bed
2.2G Sep  1 23:24 /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/annotation_fragment/Input5/chr21.bed
13.9


In [17]:
cat > motif_marginal.R << 'EOF'
### set env
source("/home/mount/project/config_sing.R")
cat("###################################################\n\n")

### Get argument: Chromomsome
ARGS  = commandArgs(trailingOnly=TRUE)
CHROM = ARGS[1]
print(CHROM)

###################################################
# Import library size
###################################################

### Helper function to get
get_sample = function(idn_sample){
    idn = idn_sample
    idn = str_replace(
        string = idn, 
        pattern = "Input[0-9]", 
        replacement = "Input")
    idn = str_replace(
        string = idn, 
        pattern = "TFX[0-9]_", 
        replacement="TFX_")
    return(idn)
}

### set path
fdiry = file.path(FD_RES, "source")
fname = "library_size.txt"
fpath = file.path(fdiry, fname)

### import library size
ctypes = c(col_integer(), col_character())
cnames = c("Size", "Fpath")
dat_lib = read_tsv(fpath, col_types=ctypes, col_names = cnames)
dat_lib = dat_lib %>% 
    mutate(Sample = tools::file_path_sans_ext(basename(Fpath))) %>%
    mutate(Group = get_sample(Sample))
dat_lib = dat_lib %>% dplyr::select(Size, Sample, Group)

###################################################
# Import annotated fragments
###################################################
cat("+++++ Import annotated fragments +++++\n")

### set column names and types
ctypes = c(col_character(), col_integer(), col_integer(), col_integer(),
           col_character(), col_integer(), col_integer(),
           col_character(), col_double(),  col_integer())
cnames = c("Chrom_Frag", "Start_Frag", "End_Frag", "Count_Frag",
           "Chrom_MTF",  "Start_MTF",  "End_MTF",
           "Motif", "Score", "Overlap")

### set samples
SAMPLES = c(
    paste0("Input", 1:5),
    paste0("TFX",   2:5, "_DMSO"),
    paste0("TFX",   2:5, "_Dex"))

### import bed files for each sample 
fdiry = file.path(FD_RES, "annotation_fragment", "filter_motif_score095")
fname = paste0(CHROM, ".bed", ".gz") 

lst_dat = lapply(SAMPLES, function(sam){
    ### set path
    fpath = file.path(fdiry, sam, fname)
    print(fpath)
    
    ### import data
    dat = read_tsv(fpath, col_types=ctypes, col_names=cnames) %>% mutate(Sample = sam)
    return(dat)
})

### arrange data
dat_ann_frag = bind_rows(lst_dat)
lst_dat = NULL

### check point: print message
head(dat_ann_frag, 3)
print(colnames(dat_ann_frag))

print("Dimension")
print(dim(dat_ann_frag))

print("Size")
print(object.size(dat_ann_frag), units="Mb")

###################################################
# Preprocess
###################################################
cat("+++++ Preprocess +++++\n")

### grouped by motif cluster and 
### split the annotated fragments into list
dat    = dat_ann_frag
lst    = dat %>% group_by(Motif) %>% group_split
motifs = lapply(lst, function(dat){unique(dat$Motif)}) %>% unlist
names(lst) = motifs

### get the list
lst_frag = lapply(lst, function(dat){
    tmp = dat %>% 
        dplyr::select(Chrom_Frag, Start_Frag, End_Frag, Count_Frag, Motif, Sample) %>%
        distinct()
    return(tmp)
})

### filter out motifs that have almost no fragment in total
cat("+++++ Preprocess: filter motifs +++++\n")
cat("Before filteration: #Motifs =", length(lst_frag), "\n")

THRESHOLD = 10
lst = lst_frag
cnt = lapply(lst, function(dat){sum(dat$Count_Frag)})
lst = lst[cnt > 10]
lst_frag = lst

cat("Threshold =", THRESHOLD, "\n")
cat("After filteration: #Motifs =", length(lst_frag), "\n")

###################################################
# Get marginal effects
###################################################
cat("+++++ Get marginal effects +++++\n")

### Helper function
### https://win-vector.com/2014/05/30/trimming-the-fat-from-glm-models-in-r/
stripGlmLR = function(cm) {
  cm$y = c()
  cm$model = c()
  
  cm$residuals = c()
  cm$fitted.values = c()
  cm$effects = c()
  cm$qr$qr = c()  
  cm$linear.predictors = c()
  cm$weights = c()
  cm$prior.weights = c()
  cm$data = c()

  cm$family$variance = c()
  cm$family$dev.resids = c()
  cm$family$aic = c()
  cm$family$validmu = c()
  cm$family$simulate = c()
  attr(cm$terms,".Environment") = c()
  attr(cm$formula,".Environment") = c()
  
  return(cm)
}

### loop through each motif estimate marginal effect
lst_res = lapply(lst_frag, function(dat){

    ### get fragments
    dat = dat %>% 
        dplyr::select(Chrom_Frag, Start_Frag, End_Frag, Count_Frag, Sample) %>%
        distinct()

    ### count
    dat = dat %>% group_by(Sample) %>% summarise(Value = sum(Count_Frag))

    ### normalize counts by library size
    tmp = dat %>% left_join(dat_lib, by="Sample")
    tmp = tmp %>%
        mutate(Norm_Value    = Value / Size) %>% 
        mutate(Lognorm_Value = log10(Value) - log10(Size)) %>%
        mutate(X = Group)

    idxs  = c("Input", "TFX_DMSO", "TFX_Dex")
    tmp$X = factor(tmp$X, levels=idxs)
    X = model.matrix(~X, tmp)
    X[,"XTFX_DMSO"] = X[,"XTFX_DMSO"] + X[,"XTFX_Dex"]
    y = tmp$Norm_Value

    ### fit model and get the summary
    fit = lm(y ~ X + 0)
    res = summary(fit)
    
    ### reduce the memory size
    res = stripGlmLR(res)
    return(res)
})

###################################################
# Save the results
###################################################

fdiry = file.path(FD_RES, "model_linear")
fname = paste0("res_marginal_", CHROM, ".rds")
fpath = file.path(fdiry, fname)
saveRDS(lst_res, fpath)
print("Done")

EOF

## Run on single chromosome

In [18]:
### set log file directory
sbatch -p ${NODE} \
    --mem=30G \
    -o ${FD_LOG}/motif_marginal_095_chrY.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
#CHROMS=($(seq 1 22) X Y)
#CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}
CHROM=chrY

###
${FD_PRJ}/sing_proj_combeffect.sh Rscript motif_marginal.R ${CHROM}

EOF

Submitted batch job 26086339


In [19]:
### set log file directory
sbatch -p ${NODE} \
    --mem=30G \
    -o ${FD_LOG}/motif_marginal_095_chr17.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
#CHROMS=($(seq 1 22) X Y)
#CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}
CHROM=chr17

###
${FD_PRJ}/sing_proj_combeffect.sh Rscript motif_marginal.R ${CHROM}

EOF

Submitted batch job 26086340


In [20]:
### set log file directory
sbatch -p ${NODE} \
    --mem=30G \
    -o ${FD_LOG}/motif_marginal_095_chr21.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
#CHROMS=($(seq 1 22) X Y)
#CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}
CHROM=chr21

###
${FD_PRJ}/sing_proj_combeffect.sh Rscript motif_marginal.R ${CHROM}

EOF

Submitted batch job 26086341


In [23]:
### set log file directory
sbatch -p ${NODE} \
    --mem=30G \
    -o ${FD_LOG}/motif_marginal_095_chr20.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
#CHROMS=($(seq 1 22) X Y)
#CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}
CHROM=chr20

###
${FD_PRJ}/sing_proj_combeffect.sh Rscript motif_marginal.R ${CHROM}

EOF

Submitted batch job 26086345


In [24]:
### set log file directory
sbatch -p ${NODE} \
    --mem=30G \
    -o ${FD_LOG}/motif_marginal_095_chrX.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
#CHROMS=($(seq 1 22) X Y)
#CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}
CHROM=chrX

###
${FD_PRJ}/sing_proj_combeffect.sh Rscript motif_marginal.R ${CHROM}

EOF

Submitted batch job 26086347


In [21]:
cat motif_marginal.R

### set env
source("/home/mount/project/config_sing.R")
cat("###################################################\n\n")

### Get argument: Chromomsome
ARGS  = commandArgs(trailingOnly=TRUE)
CHROM = ARGS[1]
print(CHROM)

###################################################
# Import library size
###################################################

### Helper function to get
get_sample = function(idn_sample){
    idn = idn_sample
    idn = str_replace(
        string = idn, 
        pattern = "Input[0-9]", 
        replacement = "Input")
    idn = str_replace(
        string = idn, 
        pattern = "TFX[0-9]_", 
        replacement="TFX_")
    return(idn)
}

### set path
fdiry = file.path(FD_RES, "source")
fname = "library_size.txt"
fpath = file.path(fdiry, fname)

### import library size
ctypes = c(col_integer(), col_character())
cnames = c("Size", "Fpath")
dat_lib = read_tsv(fpath, col_types=ctypes, col_names = cnames)
dat_lib = dat_lib %>% 
    mutate(Sample = tools::file_path_sans_ext

## Run on all chromsomes

In [11]:
### set log file directory
sbatch -p ${NODE} \
    --array=0-23 \
    --mem=20G \
    -o ${FD_LOG}/motif_marginal_095.%a.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
CHROMS=($(seq 1 22) X Y)
CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}

###
${FD_PRJ}/sing_proj_combeffect.sh Rscript motif_marginal.R ${CHROM}

EOF

Submitted batch job 26086300


In [16]:
ls -1 ${FD_RES}/model_linear

fit_interactive_target_PER1.rds
fit_marginal_target_PER1.rds
res_interactive_chr20.rds
res_interactive_chr21.rds
res_interactive_chrX.rds
res_interactive_chrY.rds
res_interactive_target_PER1.rds
res_marginal_chr13.rds
res_marginal_chr14.rds
res_marginal_chr15.rds
res_marginal_chr17.rds
res_marginal_chr18.rds
res_marginal_chr20.rds
res_marginal_chr21.rds
res_marginal_chr4.rds
res_marginal_chrX.rds
res_marginal_chrY.rds


In [14]:
tail -n 2 ${FD_LOG}/motif_marginal_095.*.txt

==> /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/log/motif_marginal_095.0.txt <==
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/TFX4_Dex/chr1.bed.gz"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/TFX5_Dex/chr1.bed.gz"

==> /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/log/motif_marginal_095.10.txt <==
/gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_STARRseq/notebooks/sing_proj_combeffect.sh: line 21: 31716 Killed                  singularity exec -H ${PWD}:/home -B ${FD_WORK}:/home/mount/work -B ${FD_PRJ}:/home/mount/project -B ${FD_RLAB}:/home/mount/reddylab ${FD_SING}/singularity_proj_combeffect.sif "$@"
slurmstepd: error: Detected 1 oom-kill event(s) in StepId=26086311.batch cgroup. Some of your processes may have been killed by the cgroup out-of-memory handler.

==> /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/log/motif_marginal_095.11.txt <==
/gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEf